In [ ]:
# Initialize GAPID environment
from helpers import gapid_env
from importlib import reload
reload(gapid_env)
import os

pathdir = globals()['_dh'][0]
env = gapid_env.gapid_env(os.path.normpath(os.path.join(pathdir, "..", "build", "RelWithDebInfo")))


In [ ]:
# Select and load trace file
from helpers import trace as gapid_trace
from importlib import reload
reload(gapid_trace)

from ipyfilechooser import FileChooser


fc = FileChooser()
if env.current_trace:
    fc.default_path = os.path.dirname(env.current_trace)
    fc.default_filename = os.path.basename(env.current_trace)

fc.filter_pattern = "*.trace"
fc.title = "<b>Select Trace</b>"
trace = None
def change_file(chooser):
    global trace
    current_trace = os.path.join(chooser.selected_path, chooser.selected_filename)
    trace =  gapid_trace.load_trace(env, current_trace)
fc.register_callback(change_file)
display(fc)

In [ ]:
# Report stats about the trace
import json
from importlib import reload
reload(gapid_trace)

cmd_stats = trace.get_stats()
print(json.dumps(cmd_stats, indent="    "))


In [ ]:
# Count the draw commands actually submitted
from collections import defaultdict

queue_submits = [i for i, v in enumerate(trace.commands) if (v["name"] == "vkQueueSubmit" and (v["tracer_flags"] & gapid_trace.MID_EXECUTION) == 0)]

submitted_draws = []
submitted_dispatches = []
for x in queue_submits:
    executed_cbs = trace.executed_commands[x]
    for y in executed_cbs:
        submitted_draws.extend([x for x in y[1] if "Draw" in trace.commands[x]["name"]])
        submitted_dispatches.extend([x for x in y[1] if "Dispatch" in trace.commands[x]["name"]])

l = defaultdict(list)
for x in submitted_draws:
    l[trace.commands[x]["name"]].append(x)

print(f"Total Draws: {len(submitted_draws)}")
for k in l.keys():
    print(f"  {k}: {len(l[k])}")

l = defaultdict(list)
for x in submitted_dispatches:
    l[trace.commands[x]["name"]].append(x)

print(f"Total Dispatches: {len(submitted_draws)}")
for k in l.keys():
    print(f"  {k}: {len(l[k])}")

In [ ]:
# Render trace on the screen
from helpers import replay
from importlib import reload
reload(replay)

create_swapchain = [x for x in trace.commands if x["name"] == "vkCreateSwapchainKHR"][0]

opts = replay.replay_options(trace, env)
#opts.use_callback_swapchain()
opts.add_layer(os.path.abspath("on_screen.cpp"), {
    "start_idx": 1,
    "width": create_swapchain["pCreateInfo"]["imageExtent"]["width"],
    "height": create_swapchain["pCreateInfo"]["imageExtent"]["height"],
}, None)

replay.replay(env, opts)

In [ ]:
# Collect screenshots from the trace (Start with the first 10 images in the trace)
screenshots_to_collect = 10
from helpers import replay
from importlib import reload
import itertools
import base64

reload(replay)

queue_submits = [i for i, v in enumerate(trace.commands) if (v["name"] == "vkQueueSubmit" and (v["tracer_flags"] & gapid_trace.MID_EXECUTION) == 0)]

idx = 0
executed_cbs = trace.executed_commands[queue_submits[idx]]
draws = []
while (len(draws) == 0):
    idx += 1
    executed_cbs = trace.executed_commands[queue_submits[idx]]

    for y in executed_cbs:
        draws.extend([(y[0], x) for x in range(len(y[1])) if "Draw" in trace.commands[y[1][x]]["name"]])

draws = itertools.groupby(draws, lambda x: x[0])
total_draws = 0
config = {"screenshot_locations": [
    {
        "submit_index": queue_submits[idx],
        "command_buffers": []
    }
]}

for x in draws:
    cbs = {
        "command_buffer": x[0],
    }
    draw_calls = list(x[1])
    draws_to_take = screenshots_to_collect - total_draws
    if len(draw_calls) < draws_to_take:
        draws_to_take = len(draw_calls)
    cbs["indices"] = [x[1] for x in draw_calls[:draws_to_take]]
    config["screenshot_locations"][0]["command_buffers"].append(cbs)
opts = replay.replay_options(env, trace)

opts.use_callback_swapchain()

def on_message(timestamp, level, message):
    if level == "Debug":
        return
    print(f'[{timestamp}] {level} :: {message}')

opts.set_message_callback(on_message)

def on_data(timestamp, data):
    print(f'{timestamp} -- {data["width"], data["height"], data["format"], len(data["data"])}')
    data = base64.b64decode(data["data"])
    # Todo: Actually display this data!

def on_layer_message(timestamp, level, message):
    if level == "Debug":
        return
    print(f'screenshot.cpp:: [{timestamp}] {level} :: {message}')

opts.add_layer(os.path.abspath("screenshot.cpp"), config, data_callback=on_data, message_callback=on_layer_message)

replay.replay(env, opts)